# Implementing Multiple Linear Regression

We are now ready to actually implement a multiple regression model from scratch using Python!

As we did in univariate linear regression, we'll start by importing two libraries: `numpy` for handling matrix computations, and `pandas` for importing, exporting and visualizing our data.

Recall our importing syntax:

In [ ]:
import numpy as np
import pandas as pd

We'll now use `pandas` to read our apartments price dataset into a `pandas` dataframe `df`.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/marciorvneto/ml-course/main/multivariate-linear-regression/apartments.csv")
df

## Visualizing our data using scatter plots

It is always a good idea to get a sense of our data before attempting to train any models on it. For high-dimensional data, a nice way of visualizing pairwise relations between features is by using a scatter plot matrix. Pandas makes this very simple for us:

In [ ]:
pd.plotting.scatter_matrix(df)

## Splitting our data into training and testing datasets

As we've discussed before, we should split our dataset into a training dataset and a testing dataset. We fit our model with the training data, and evaluate its performance using data that it's never seen before - the test data.

We'll proceed as before, by doing a 70%/30% split on our data:

In [ ]:
percent_training = 0.7
percent_test = 1 - percent_training

As we've mentioned in our last demonstration, it's good practice to shuffle our training data to break possible sequential correlations between our data points.

In [ ]:
num_total_data = len(df)                                  # Total number of data points
num_training_data = int(percent_training*num_total_data)  # Number of training data points
num_test_data = num_total_data - num_training_data        # Number of test data points

indices = np.arange(num_total_data)                       # Create an array of indices for our data
np.random.shuffle(indices)                                # Shuffle the indices
indices

Notice that the indices have been permuted. Now, all we've got left to do is select the first `num_training_data` indices as our training indices, and select the remaining ones to be our test indices:

In [ ]:
training_indices = indices[:num_training_data]
test_indices = indices[num_training_data:]

# Building the design matrix

Recall that in order to find the multiple linear regression coefficients $\hat{\beta}$, we need to solve the following system of equations, also known as the *normal equations*:

$$(\mathbb{X}^T\mathbb{X})\hat{\beta} = \mathbb{X}^T\mathbb{Y}$$

The matrix $\mathbb{X}$ is the so-called *design matrix*. Recall that its first column is made of ones, and each subsequent column corresponds to a feature of our model.

Numpy and pandas make creating this matrix quite straightforward. We first convert our pandas dataframe to a numpy array `dataset`:

In [ ]:
dataset = df.to_numpy()

We can now separate our training data from our test data by taking the corresponding rows from `dataset`:

In [ ]:
training_data = dataset[training_indices]
test_data = dataset[test_indices]

`training_data` and `test_data` are two numpy matrices containing our train and test data. Now let's create the design matrix using our training data.

The first thing we need is a column of ones. Numpy's function `ones` allows us to create an array of ones of whichever shape we like. We need a column of ones with `num_training_data` elements:

In [ ]:
col_ones = np.ones((num_training_data,1))
col_ones

The next columns correspond to our features. By inspecting our data, we notice that the first and second columns (indices 0 and 1, in Python talk) correspond to area and distance. The third column (index 2), on the other hand, corresponds to the price, our label.

Therefore, we only need to include columns from 0 to 2 (not including). In Python, we can isolate them like this:

In [ ]:
features = training_data[:, 0:2]   # 0:2 means 0 and 1, 2 is not included
features

Now we need to **horizontally stack** these columns to complete our design matrix. We do that using the funciton `hstack`:

In [ ]:
X = np.hstack([col_ones, features])
X

Nice! Let us now take the last column of our training data to get our prices:

In [ ]:
Y = training_data[:,-1]    # This is equivalent to training_data[:,2]. "-1" means "last index"
Y

## Solving for the model parameters

All we need to do now is solve the linear system corresponding to the normal equations.

We're in luck, because `numpy` is excellent at solving linear systems! In fact, we can solve very large systems very quickly and with great precision.

In order to leverage `numpy`'s system solving capabilities, we need to express our linear system as:

$$ A\mathbf{x} = \mathbf{b} $$

Where $A$ is a matrix of constant coefficients, $\mathbf{b}$ is a vector (or column matrix) of constants, and $\mathbf{x}$ is our vector of unknowns.

Let's compare this equation with the normal equations:

$$(\mathbb{X}^T\mathbb{X})\hat{\beta} = \mathbb{X}^T\mathbb{Y}$$

Our vector of unknowns here are the $\hat{\beta}$. Likewise, we find that:

* $A = \mathbb{X}^T\mathbb{X}$
* $\mathbf{b} = \mathbb{X}^T\mathbb{Y}$

Let's see how we can build  $A$ and $\mathbf{b}$ using `numpy`:

In [ ]:
A = X.T.dot(X)           # Transposes X and multiplies it by X
b = X.T.dot(Y)           # Transposes X and multiplies it by Y

Solving the system for $\hat{\beta}$ is now as simple as calling a function. To be more precise, we'll be calling the function `solve`, which is stored in the linear algebra module `linalg` of `numpy`:

In [ ]:
beta_hat = np.linalg.solve(X.T.dot(X), X.T.dot(Y))
beta_hat

## Evaluating our model's performance

As before, in order to evaluate how well our model should perform on real-world data, we'll calculate the mean squared error, MSE, using our test data, which we expect to be a good proxy to real world data.

Let us begin by defining a function `mse`:

In [ ]:
def mse(beta, x, y_real):
    y_predicted = x.dot(beta)
    error = np.mean((y_real - y_predicted)**2)
    return error

Let's see how well we're doing when we plug in our test data:

In [ ]:
X_test = np.hstack([np.ones((num_test_data,1)), test_data[:,0:2]])
Y_test = test_data[:,-1]

test_error = mse(beta_hat, X_test, Y_test)
test_error

## Graphically visualizing our model's quality

It's nice to get a sense of how well our model behaves in the test data by plotting the predicted prices versus their actual values.

If our predictions were perfect, all our values would line up along the line $y = x$. In practice, however, this is not the case. The more our predicted points deviate from this line, the worse our predictions are.

Let's see how we can visualize our data using `matplotlib`:

In [ ]:
import matplotlib.pyplot as plt

Y_predicted = X_test.dot(beta_hat)

plt.plot(Y_test, Y_predicted, 'o')
x = np.linspace(np.min(Y_test), np.max(Y_test))

plt.plot(x,x)
plt.grid()
plt.legend(["Predicted values", "y=x"])

Not bad at all, but there's room for improvement. In the next lectures, we'll see how we can leverage our 